In [ ]:
# this is to get the access to the googel drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# this is to goto the directory where i stored the mask rcnn as in this code i use it for detection
import os
os.chdir("/content/drive/My Drive/auto_parking_projects/Mask_RCNN/")

In [ ]:
# this is to save the files generated during the processing
!mkdir saved_images

In [ ]:
import numpy as np # this is to get the calculations
import matplotlib as pt # for plotting facilities
import cv2 # for capture handling
# these are from the rcnn
import mrcnn.config 
import mrcnn.utils
from mrcnn.model import MaskRCNN

from pathlib import Path

In [ ]:
!nvidia-smi # this is to check about the gpu 

In [ ]:
# define the configurations
class MaskRCNNConfig(mrcnn.config.Config):
    NAME = "coco_pretrained_model_config"
    IMAGES_PER_GPU = 1
    GPU_COUNT = 1
    NUM_CLASSES = 1 + 80  
    DETECTION_MIN_CONFIDENCE = 0.6 #setted to 60%

# Root dir --- this is the Mask_RCNN folder and in there mask_rcnn_coco.h5 present and do all the things in this folder
ROOT_DIR = Path(".")
#Trained model loc
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
if not os.path.exists(COCO_MODEL_PATH):
    mrcnn.utils.download_trained_weights(COCO_MODEL_PATH)

VIDEO_SOURCE = "resources/parking_video.mp4" # here is the videos and photos for the training are included and this folder should be in the root

# configurations for the model
model = MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=MaskRCNNConfig())
model.load_weights(COCO_MODEL_PATH, by_name=True)

video_capture = cv2.VideoCapture(VIDEO_SOURCE) # getting the video captures using opencv

parked_car_boxes = None
free_space_frames = 0
count = 0
free_space = False

def get_car_boxes(boxes, class_ids):
    # when detecting objects related to any of the classes are sent to this and purpose of this function is returning 
    # the boxes related to the class_ids[3,8,6] which are related to vehicles
    car_boxes = []
    # here what come is any type of object and from here we only get the bounding boxes for only vehicles
    for i, box in enumerate(boxes):
        if class_ids[i] in [3, 8, 6]:
            car_boxes.append(box)
    return np.array(car_boxes)

while video_capture.isOpened():
    success, frame = video_capture.read()# if read the capture one then it is succesful or not and the frame(=image) taken
    
    if not success:
        print("couldn't read video")
        break
        
    elif count<80:
        # by increasing and decreasing the checking counter value we can tune the place where the green box is placed
        # this counter is to get the time gap that we are checking the free space of the park
        success, frame2 = video_capture.read() #this is to get another read and compare the motion of vehicles
        # --------------------------------------------------------------------------------------------------------------------------------------
        # this set of codes is used to track moving objects -- to get an idea of these functions prefer some documentations
        # --------------------------------------------------------------------------------------------------------------------------------------
        d = cv2.absdiff(frame, frame2)# then get the absolute difference of two frames to identify the differences of pixels
        grey = cv2.cvtColor(d, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(grey, (1, 1), 0)
        ret, th = cv2.threshold( blur, 20, 255, cv2.THRESH_BINARY)
        #morphological operations to remove the unnecessary parts in the getting processed image
        dilated = cv2.dilate(th, np.ones((10, 10), np.uint8), iterations=1 )
        eroded = cv2.erode(dilated, np.ones((10, 10), np.uint8), iterations=1 )
        #fill the contours for even a better morphing of the vehicle
        c, h = cv2.findContours(eroded, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)# c-contour h-hierachy 
        # contour is taken and it is applied to original image and get the moving object only one object is getting the the contour in one image
        frame2 = cv2.drawContours(frame2, c, -1, (0,0,0), cv2.FILLED)
        # --------------------------------------------------------------------------------------------------------------------------------------
        if count%5 == 0:
            # from every 5 time show the images with contours for the cars which are moving
            print("Current frame counter" + str(count))
            pt.pyplot.imshow(frame2)
            pt.pyplot.show()
        
        count = count + 1# moving object detection is happen for 40
        continue
    
    rgb_image = frame[:, :, ::-1] # this is how convert gbr to rgb(opencv use gbr but to feed to mrcnn use rgb)
    # The r variable will now have the results of detection:
    # - r['rois'] are the bounding box of each detected object
    # - r['class_ids'] are the class id (type) of each detected object
    if parked_car_boxes is None:
        # this happens for the very first time after 40 times--- for first frame getting the initial places of cars
        results = model.detect([rgb_image], verbose=0)# then given to the model
        r = results[0]
        # This is the first frame of video - assume all the cars detected are in parking spaces.
        # Save the location of each car as a parking space box and go to the next frame of video.
        parked_car_boxes = get_car_boxes(r['rois'], r['class_ids'])
          
    elif count%200 == 0 and len(parked_car_boxes) != 0:
        # get the places where the cars are located
        results = model.detect([rgb_image], verbose=0)# then given to the model
        r = results[0]
        car_boxes = get_car_boxes(r['rois'], r['class_ids'])

        # then take the previous one and compare the overlapping with it
        overlaps = mrcnn.utils.compute_overlaps(parked_car_boxes, car_boxes)

        # for every parking area
        for parking_area, overlap_areas in zip(parked_car_boxes, overlaps):

            #picrure ivided into parts and take the overlap therefore it sends array
            max_IoU_overlap = np.max(overlap_areas)

            # at every time get the the area of parking place of the car
            y1, x1, y2, x2 = parking_area

            # if there is no car in that place where there was a car before
            if max_IoU_overlap < 0.15:
                # if there is no vehicle in that car slot then draw a box around it in green color
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
                free_space = True
            else:
                # if there is a vehicle in that slot then draw a box of red color
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 1)

            # Write the IoU measurement inside the box -- just opencv
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, f"{max_IoU_overlap:0.2}", (x1 + 6, y2 - 6), font, 0.3, (255, 255, 255))

        # If a space has been free for several frames, pretty sure it is really free
        if free_space:
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, f"PARKING SLOT IS AVAILABEL!", (10, 150), font, 3.0, (0, 255, 0), 2, cv2.FILLED)

    #saving each frame
    name = str(count) + ".jpg"
    name = os.path.join('./saved_images', name)
    cv2.imwrite(name, frame)
    count+=1
    
    #'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
print("Video finished")
video_capture.release()

In [ ]:
import glob
# this code is usually for creating a video from set of photos
images = list(glob.iglob(os.path.join('./saved_images', '*.*')))
images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))

def make_video(outvid, images=None, fps=30, size=None,is_color=True, format="FMP4"):
   
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    for image in images:
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)
    vid.release()
    return vid
  
make_video('./detecting.mp4', images, fps=30) 